Creating the DB - FAISS

In [2]:
!pip install langchain
!pip install transformers
!pip install pydantic
!pip install fastapi

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import json

# Load preprocessed data
with open("datasets/pqaa_documents.json", "r") as f:
    documents = json.load(f)

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vectorstore
texts = [doc["text"] for doc in documents]
metadatas = [doc["metadata"] for doc in documents]
vectorstore = FAISS.from_texts(texts, embedding=embedding_model, metadatas=metadatas)

# Save FAISS index
vectorstore.save_local("db/faiss")


C:\Users\SNR\AppData\Local\Temp\ipykernel_13076\3116351894.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\SNR\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RAG - Pipeline

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load LLM
model_name = "EleutherAI/gpt-neo-2.7B"  # Replace with desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Wrap LLM with LangChain
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Load vectorstore
from langchain.vectorstores import FAISS  # Or Chroma if used
vectorstore = FAISS.load_local("db/faiss")  # Replace with ChromaDB if needed
retriever = vectorstore.as_retriever()

# Create RAG pipeline
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

# Test the pipeline
query = "What are the symptoms of diabetes?"
response = rag_chain.run(query)
print("Answer:", response["answer"])
print("Sources:", response["source_documents"])


RAG FastAPI

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

# Define input schema
class Query(BaseModel):
    question: str

@app.post("/query/")
async def query_rag(query: Query):
    response = rag_chain.run(query.question)
    return {"answer": response["answer"], "sources": response["source_documents"]}


In [ ]:
uvicorn app:app --reload

Deploy on Hugging Face

In [ ]:
git add .
git commit -m "Add RAG pipeline"
git push
